In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd
from linkedin_bot import *

In [3]:
if not os.path.exists("data"):
    os.makedirs("data")

driver = webdriver.Chrome(service=Service(executable_path="./chromedriver"))

In [4]:
email = "kater44555@elastit.com"
password = "kater44555@elastit"

if os.path.exists("data/cookies.txt"):
    driver.get("https://www.linkedin.com/")
    driver = load_cookie(driver, "data/cookies.txt")
    driver.get("https://www.linkedin.com/")
else:
    driver = login(driver, email, password)
    save_cookie(driver, "data/cookies.txt")

In [5]:
driver.get(url='https://www.linkedin.com/jobs/search/?keywords="data scientist"&location=Singapore&sortBy=DD')
time.sleep(DELAY)

# OR

# driver = search_linkedin(driver, keywords='"data scientist"', location='Singapore')


In [ ]:
job_data = pd.DataFrame(columns=['job_id', 'metadata', 'job_description'])
job_data.to_csv("data/job_data.csv", encoding='utf-8', index=False, mode="a", header=True)

for page in range(2, 24):
    time.sleep(DELAY)
    jobs = driver.find_elements(By.CLASS_NAME, "occludable-update")

    for job in jobs:
        driver = scroll_to(driver, job)
        time.sleep(DELAY)
        data = get_job_data(driver, job)
        job_data = job_data.append(data, ignore_index=True)
        job_data.iloc[[-1]].to_csv("data/job_data.csv" , encoding='utf-8', index=False, mode="a", header=False)
        time.sleep(DELAY)
    
    driver.find_element(By.XPATH, f"//button[@aria-label='Page {page}']").click()
    time.sleep(DELAY)

In [33]:
# driver.close()

In [16]:
job_data = job_data.drop_duplicates(subset=['job_id', 'job_description']).reset_index(drop=True)

In [20]:
job_data['metadata_len'] = job_data.metadata.apply(lambda x: len(x))

In [30]:
job_data.query("metadata_len == 9").sample(1).metadata.values

array([list(['#SGUnitedTraineeships - Junior Data Scientist Traineeship', 'TOLUNA HONG KONG LIMITED SINGAPORE BRANCH', 'Singapore, Singapore', '6 days ago', 'Full-time · Entry level', 'See recent hiring trends for TOLUNA HONG KONG LIMITED SINGAPORE BRANCH. Try Premium for free', 'Apply', 'Save', 'Save #SGUnitedTraineeships - Junior Data Scientist Traineeship at TOLUNA HONG KONG LIMITED SINGAPORE BRANCH'])],
      dtype=object)

In [31]:
job_data.query("metadata_len == 14").sample(1).metadata.values

array([list(['Data Scientist', 'Toptal', 'Singapore, Singapore', 'Remote', '9 hours ago', '2 applicants', 'Contract · Mid-Senior level', '1,001-5,000 employees · Internet', '1 school alumni', 'See recent hiring trends for Toptal - Second Feed. Try Premium for free', 'Actively recruiting', 'Apply', 'Save', 'Save Data Scientist at Toptal'])],
      dtype=object)